In [2]:
import requests
from bs4 import BeautifulSoup as bs
import types
import json
import pymysql
import time
import sys

db = pymysql.connect('140.138.77.90','knkn','tp6bjo4u;6','knkn',use_unicode=True, charset="utf8mb4")
cursor = db.cursor()
sql="SELECT `user_id` FROM `Questions`"
cursor.execute(sql)
results = cursor.fetchall()

for x in range(len(results)):#for x in results
    
    owner_id = results[x][0]
    sql="SELECT `user_id`  FROM `User` WHERE `user_id` = "+str(owner_id)#把我從question抓下來的user_id，去User table找一樣的放進res
    cursor.execute(sql)
    res=cursor.fetchall()
    if not res: #若這個user_id不在User table裡面，就開始做事
        print(owner_id,' hasn\'t recorded')
        
        #-----------------------Start---------------------------
        user_url = "https://stackoverflow.com/users/" + str(owner_id)
        user_page = bs(requests.get(user_url).text,'html.parser')

        reputation = user_page.find('div',class_='reputation').text.split(' r')[0]#可能大於一千，就會有,不能直接轉型
        reputation = int(reputation.strip().replace(',',''))

        answers = int(user_page.find('div',class_='stat answers col-3').text.split('a')[0].strip().replace(',',''))

        questions = int(user_page.find('div',class_='stat questions col-3').text.split('q')[0].strip().replace(',',''))

        reached = user_page.find('div',class_='stat people-helped col-5').text.split('p')[0].strip().replace('~','').replace(',','')
        if(reached[-1] == 'k'):
            reached = float(reached.replace('k',''))
            reached = reached * 1000
        elif(reached[-1] == 'm'):
            reached = float(reached.replace('m',''))
            reached = reached * 1000000
        else:
            reached = float(reached)

        profile = user_page.find('ul',class_='list-unstyled')
        location = ''
        age = ''
        views = ''
        for x in profile.find_all('li'):
            if(x.find('svg',class_='svg-icon iconLocation') is not None):#代表這傢伙有居住地
                location = x.text.strip()
            if(x.find('svg',class_='svg-icon iconHistory') is not None):#這傢伙的帳號創建日
                age = x.find('span')['title'].split('Z')[0]
            if(x.find('svg',class_='svg-icon iconEye') is not None):
                views = int(x.text.split(' p')[0].strip().replace(',',''))
                
        #---------Pass the values in database-----------
        cursor.execute('INSERT INTO User (user_id,reputation,answers,questions,reached,location,age,views) VALUES('
                       + str(owner_id) + ',' + str(reputation) + ',' + str(answers) + ',' + str(questions) + ',' + str(reached) + ',"' + location + '","' + age + '",' + str(views) + ')')
        db.commit()
        #-----------------End User table----------------
        
        #---------------Start UserTag table-------------
        page = 1
        total_count = 0
        total_score = 0

        while True:
            try:
                toptags_url = 'https://api.stackexchange.com/2.2/users/'+str(owner_id)+'/top-tags?page=' + str(page) + '&pagesize=100&site=stackoverflow'
                user_toptags = bs(requests.get(toptags_url).text,'html.parser')
                json_str = json.loads(user_toptags.text)

                for x in range(len(json_str['items'])):
                    total_count = json_str['items'][x]['answer_count'] + json_str['items'][x]['question_count']#把answer和question的數量加起來
                    total_score = json_str['items'][x]['answer_score'] + json_str['items'][x]['question_score']#把answer和question的分數加起來
                    average = total_score/total_count#算平均每個問題得到幾分

                    #---------Pass the values in database-----------
                    cursor.execute('INSERT INTO UserTag (user_id,tag_name,answer_count,answer_score,question_count,question_score,total_count,total_score,average) VALUES('
                                           +str(owner_id)+',"'+str(json_str['items'][x]['tag_name'])+'",'+str(json_str['items'][x]['answer_count'])+','+str(json_str['items'][x]['answer_score'])+','+str(json_str['items'][x]['question_count'])+','+str(json_str['items'][x]['question_score'])+','+str(total_count)+','+str(total_score)+','+str(average)+')')
                    db.commit()
                break
                #if (json_str['has_more'] is False):#還沒結束，後面還有
                    #break
                #page = page + 1
                
            except:#在上面的程式碼如果進去api找不到資料
                print("!!!!API Explosion!!!!")
                db.close()
                sys.exit(0)#API爆炸，程式強制被結束
        #------------End UserTag table------------
        
        #------------Start User_ Table------------
        sql="SELECT COUNT(user_id) FROM Comments WHERE user_id = "+str(owner_id)
        cursor.execute(sql)
        counts = cursor.fetchall()
        comments_counts=counts[0][0]
        
                                   
        sql="SELECT COUNT(user_id) FROM Answers WHERE user_id = "+str(owner_id)
        cursor.execute(sql)
        counts = cursor.fetchall()
        answers_counts=counts[0][0]
    
        sql="SELECT COUNT(user_id) FROM Questions WHERE user_id = "+str(owner_id)
        cursor.execute(sql)
        counts = cursor.fetchall()
        questions_counts=counts[0][0]
                                   
        sql="SELECT COUNT(is_answered) FROM Questions WHERE user_id = "+str(owner_id) + " and is_answered = 1"
        cursor.execute(sql)
        counts = cursor.fetchall()
        questions_answered=counts[0][0]
        
        sql="SELECT COUNT(is_accepted) FROM Answers WHERE user_id = "+str(owner_id) + " and is_accepted = 1"
        cursor.execute(sql)
        counts = cursor.fetchall()
        answers_accepted=counts[0][0]
                
        sql="SELECT score FROM Questions WHERE user_id = "+str(owner_id)
        cursor.execute(sql)
        counts = cursor.fetchall()
        questions_score = 0
        for y in range(len(counts)):
            questions_score = questions_score + counts[y][0]
        
        sql="SELECT score FROM Answers WHERE user_id = "+str(owner_id)
        cursor.execute(sql)
        counts = cursor.fetchall()
        answers_score = 0
        for y in range(len(counts)):
            answers_score = answers_score + counts[y][0]
        
        #---------Pass the values in database-----------
        cursor.execute("INSERT INTO User_ (user_id, comments, answers, questions, questions_answered, answers_accepted, questions_score, answers_score) VALUES ("
               +str(owner_id)+','+str(comments_counts)+','+str(answers_counts)+','+str(questions_counts)+','+str(questions_answered)+','+str(answers_accepted)+','+str(questions_score)+','+str(answers_score)+')')
        db.commit()
        
    else:#若這個user_id在User table裡面
        print(owner_id,"!!!!This id has been recoreded.!!!!")

db.close()

1275169 !!!!This id has been recoreded.!!!!
3071284  hasn't recorded
4631427  hasn't recorded
8465774  hasn't recorded
172885  hasn't recorded
5601284  hasn't recorded
3728901  hasn't recorded
2435473  hasn't recorded
4243687  hasn't recorded
4927984  hasn't recorded
4885801  hasn't recorded
719662  hasn't recorded
4022390  hasn't recorded
1868008  hasn't recorded
1590389  hasn't recorded
!!!!API Explosion!!!!


SystemExit: 0

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
